# item2itemで生成する候補の元々のitemはどのくらい取ればよいかの実験
## 結論
* repurchase 自体は際限なく取ったほうが良い（恐らく遠い昔に買ったユーザーも取り込むため）
* cooc とか same_product_code とかそういうのは、期間は長め（多分↑と同じ理由で）だが直近&量でソートした100件くらいで良さそう
  * cooc は現状あんまり機能していない感じはするので、LightFMかLightGCNで置き換えるか、ルールをちゃんとする（今のはChrisのやつと等価？）
* 計算時間との兼ね合いを考えると、repurchase は全区間についてとり、item2item系のやつは直近＆量で100件くらい残すのがよい？

In [4]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from metric import apk, mapk
from utils import plot_images
import catboost

In [5]:
def objective(trial):
    num_popular_items = 12
    num_train_weeks = 3
    volume_week = 1
    transaction_dynamic_week = 8
    num_cooc_week = 12
    cooc_prob = 0.03
    num_repurchase_week = trial.suggest_int('num_repurchase_week', 3, 16)
    repurchase_rank = trial.suggest_int('repurchase_rank', 1, 400)
    base_cands_cooc_week = trial.suggest_int('base_cands_cooc_week', 1, 16)
    base_cands_cooc_rank = trial.suggest_int('base_cands_cooc_rank', 1, 400)
    base_cands_same_week = trial.suggest_int('base_cands_same_week', 1, 16)
    base_cands_same_rank = trial.suggest_int('base_cands_same_rank', 1, 400)

    dataset = '100'

    transactions = pd.read_pickle(f"input/{dataset}/transactions_train.pkl")
    users = pd.read_pickle(f"input/{dataset}/users.pkl")
    items = pd.read_pickle(f"input/{dataset}/items.pkl")

    def create_candidates(transactions: pd.DataFrame, target_users: np.ndarray, week: int) -> pd.DataFrame:
        """
        transactions
            original transactions (user, item, week)
        target_users, week
            候補生成対象のユーザー
            weekで指定されている週の段階での情報のみから作られる
        """
        assert len(target_users) == len(set(target_users))

        def create_candidates_repurchase(
                strategy: str,
                transactions: pd.DataFrame,
                target_users: np.ndarray,
                week_start: int,
                week_end: int,
                rank_threshold: int,
                ) -> pd.DataFrame:
            tr = transactions.query("user in @target_users and @week_start <= week <= @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)

            gr_week = tr.groupby(['user', 'item'])['week'].min().reset_index(name='week')
            gr_volume = tr.groupby(['user', 'item']).size().reset_index(name='volume')

            gr_week['week_rank'] = gr_week.groupby('user')['week'].rank()
            gr_volume['volume_rank'] = gr_volume.groupby('user')['volume'].rank()

            candidates = gr_week.merge(gr_volume, on=['user', 'item'])
            candidates['rank'] = 1000000 * candidates['week'] - candidates['volume']
            candidates['rank'] = candidates.groupby('user')['rank'].rank()
            candidates = candidates.query("rank <= @rank_threshold").reset_index(drop=True)
            candidates = candidates[['user', 'item', 'week_rank', 'volume_rank']]
            candidates = candidates.rename(columns={'week_rank': f'{strategy}_week_rank', 'volume_rank': f'{strategy}_volume_rank'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_popular(
                strategy: str,
                transactions: pd.DataFrame,
                target_users: np.ndarray,
                week_start: int,
                week_end: int) -> pd.DataFrame:
            tr = transactions.query("@week_start <= week <= @week_end")[['user', 'item']].drop_duplicates(ignore_index=True)
            popular_items = tr['item'].value_counts().index.values[:num_popular_items]
            popular_items = pd.DataFrame({
                'item': popular_items,
                'rank': range(num_popular_items),
                'crossjoinkey': 1,
            })

            candidates = pd.DataFrame({
                'user': target_users,
                'crossjoinkey': 1,
            })

            candidates = candidates.merge(popular_items, on='crossjoinkey').drop('crossjoinkey', axis=1)
            candidates = candidates.rename(columns={'rank': f'{strategy}_rank'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_cooc(
                strategy: str,
                transactions: pd.DataFrame,
                week_start: int,
                week_end: int,
                base_candidates: pd.DataFrame) -> pd.DataFrame:
            hoge = cooc_prob
            tr = transactions.query("@week_start <= week == @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)
            tr = tr.merge(tr.rename(columns={'item': 'item_with', 'week': 'week_with'}), on='user').query("item != item_with and week <= week_with")[['item', 'item_with']].reset_index(drop=True)
            gr_sz = tr.groupby('item').size().reset_index(name='tot')
            gr_cnt = tr.groupby(['item', 'item_with']).size().reset_index(name='cnt')
            item2item = gr_cnt.merge(gr_sz, on='item')
            item2item['ratio'] = item2item['cnt'] / item2item['tot']
            item2item = item2item.query("ratio >= @hoge").reset_index(drop=True)

            base_candidates_columns = [c for c in base_candidates.columns if '_' in c]
            replace = {c: f"{strategy}_{c}" for c in base_candidates_columns}
            candidates = base_candidates.merge(item2item, on='item').drop(['item', 'cnt'], axis=1).rename(columns={'item_with': 'item'}).rename(columns=replace)
            candidates = candidates.rename(columns={'ratio': f'{strategy}_ratio', 'tot': f'{strategy}_tot'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_same_product_code(
                strategy: str,
                items: pd.DataFrame,
                base_candidates: pd.DataFrame) -> pd.DataFrame:
            item2item = items[['item', 'product_code']].merge(items[['item', 'product_code']].rename({'item': 'item_with'}, axis=1), on='product_code')[['item', 'item_with']].query("item != item_with").reset_index(drop=True)

            base_candidates_columns = [c for c in base_candidates.columns if '_' in c]
            replace = {c: f"{strategy}_{c}" for c in base_candidates_columns}
            candidates = base_candidates.merge(item2item, on='item').drop('item', axis=1).rename(columns={'item_with': 'item'}).rename(columns=replace)

            candidates['strategy'] = strategy
            return candidates

        candidates_repurchase = create_candidates_repurchase('repurchase', transactions, target_users, week, week + num_repurchase_week, repurchase_rank)
        candidates_popular = create_candidates_popular('pop', transactions, target_users, week, week)

        hoge = create_candidates_repurchase('base_cooc', transactions, target_users, week, week + base_cands_cooc_week, base_cands_cooc_rank)
        fuga = create_candidates_repurchase('base_same', transactions, target_users, week, week + base_cands_same_week, base_cands_same_rank)

        candidates_cooc = create_candidates_cooc('cooc', transactions, week, week + num_cooc_week, hoge)
        candidates_same_product_code = create_candidates_same_product_code('same_product_code', items, fuga)

        candidates = [
            candidates_repurchase,
            candidates_popular,
            candidates_cooc,
            candidates_same_product_code,
        ]

        candidates = pd.concat(candidates).reset_index(drop=True)

        volumes = candidates.groupby('strategy').size().reset_index(name='volume').sort_values(by='volume', ascending=False).reset_index(drop=True)
        volumes['ratio'] = volumes['volume'] / volumes['volume'].sum()

        return candidates

    # valid: week=1
    # train: week=2..1+num_train_weeks
    candidates = []
    for week in range(1, 2+num_train_weeks):
        target_users = transactions.query("week == @week")['user'].unique()
        candidates.append(create_candidates(transactions, target_users, week+1))
    def merge_labels(candidates: pd.DataFrame, week: int) -> pd.DataFrame:
        """
        candidatesに対してweekで指定される週のトランザクションからラベルを付与する
        """
        labels = transactions[transactions['week'] == week][['user', 'item']].drop_duplicates(ignore_index=True)
        labels['y'] = 1
        original_positives = len(labels)
        labels = candidates.merge(labels, on=['user', 'item'], how='left')
        labels['y'] = labels['y'].fillna(0)

        remaining_positives_total = labels[['user', 'item', 'y']].drop_duplicates(ignore_index=True)['y'].sum()
        recall = remaining_positives_total / original_positives

        volumes = candidates.groupby('strategy').size().reset_index(name='volume')
        remaining_positives = labels.groupby('strategy')['y'].sum().reset_index()
        remaining_positives = remaining_positives.merge(volumes, on='strategy')
        remaining_positives['recall'] = remaining_positives['y'] / original_positives
        remaining_positives['hit_ratio'] = remaining_positives['y'] / remaining_positives['volume']
        remaining_positives = remaining_positives.sort_values(by='y', ascending=False).reset_index(drop=True)

        return labels

    for idx in range(len(candidates)):
        candidates[idx] = merge_labels(candidates[idx], 1+idx)

    def get_query_group(df):
        def run_length_encoding(sequence):
            comp_seq_index, = np.concatenate(([True], sequence[1:] != sequence[:-1], [True])).nonzero()
            return sequence[comp_seq_index[:-1]], np.ediff1d(comp_seq_index)
        users = df['user'].values
        _, group = run_length_encoding(users)
        return list(group)

    def drop_trivial_users(labels):
        """
        LightGBMのxendgcやlambdarankでは正例のみや負例のみのuserは学習に無意味なのと、メトリックの計算がおかしくなるので省く
        """
        bef = len(labels)
        df = labels[labels['user'].isin(labels[['user', 'y']].drop_duplicates().groupby('user').size().reset_index(name='sz').query("sz==2").user)].reset_index(drop=True)
        aft = len(df)
        return df

    for idx in range(len(candidates)):
        candidates[idx]['week'] = 1+idx

    valid_all = candidates[0].copy()

    for idx in range(len(candidates)):
        candidates[idx] = drop_trivial_users(candidates[idx])

    def attach_features(transactions: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame, candidates: pd.DataFrame, week: int) -> pd.DataFrame:
        """
        user, itemに対して特徴を横付けする
        week: これを含めた以前の情報は使って良い
        """
        n_original = len(candidates)
        df = candidates.copy()

        # user static features
        user_features = ['FN', 'Active', 'age', 'club_member_status_idx', 'fashion_news_frequency_idx']
        df = df.merge(users[['user'] + user_features], on='user')

        # item static features
        item_features = [c for c in items.columns if c.endswith('idx')]
        df = df.merge(items[['item'] + item_features], on='item')

        # user dynamic features (transactions)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('user')[['price', 'sales_channel_id']].agg(['mean', 'std'])
        tmp.columns = ['user_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='user', how='left')

        # item dynamic features (transactions)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('item')[['price', 'sales_channel_id']].agg(['mean', 'std'])
        tmp.columns = ['item_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='item', how='left')

        # item dynamic features (user features)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").merge(users[['user', 'age']], on='user')
        tmp = tmp.groupby('item')['age'].agg(['mean', 'std'])
        tmp.columns = [f'age_{a}' for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='item', how='left')

        # item freshness features
        tmp = transactions.query("@week <= week").groupby('item')['week'].min().reset_index(name='item_week_min')
        tmp['item_week_min'] -= week
        df = df.merge(tmp, on='item', how='left')

        # item volume features
        num_weeks = volume_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('item').size().reset_index(name='item_volume')
        df = df.merge(tmp, on='item', how='left')

        # # user freshness features
        # tmp = transactions.query("@week <= week").groupby('user')['week'].min().reset_index(name='user_week_min')
        # tmp['user_week_min'] -= week
        # df = df.merge(tmp, on='user', how='left')

        # # user volume features
        # num_weeks = 1
        # week_end = week + num_weeks
        # tmp = transactions.query("@week <= week < @week_end").groupby('user').size().reset_index(name='user_volume')
        # df = df.merge(tmp, on='user', how='left')

        # user-item freshness features
        tmp = transactions.query("@week <= week").groupby(['user', 'item'])['week'].min().reset_index(name='user_item_week_min')
        tmp['user_item_week_min'] -= week
        df = df.merge(tmp, on=['item', 'user'], how='left')

        # user-item volume features
        num_weeks = volume_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby(['user', 'item']).size().reset_index(name='user_item_volume')
        df = df.merge(tmp, on=['user', 'item'], how='left')

        assert len(df) == n_original
        return df


    valid_all = attach_features(transactions, users, items, valid_all, 2)
    for idx in range(len(candidates)):
        candidates[idx] = attach_features(transactions, users, items, candidates[idx], 2+idx)

    for idx in range(len(candidates)):
        candidates[idx]['group'] = candidates[idx]['week'].astype(str) + '_' + candidates[idx]['user'].astype(str)
        candidates[idx] = candidates[idx].sort_values(by='group').reset_index(drop=True)

    valid = candidates[0]

    def concat_train(datasets, begin, num):
        train = pd.concat([datasets[idx] for idx in range(begin, begin+num)])
        return train

    train = concat_train(candidates, 1, num_train_weeks)
    feature_columns = [c for c in valid.columns if c not in ['y', 'strategy', 'week', 'group']]


    cat_feature_values = [c for c in feature_columns if c.endswith('idx')]
    cat_features = [feature_columns.index(c) for c in cat_feature_values]

    train_pool = catboost.Pool(data=train[feature_columns], label=train['y'], group_id=train['group'], cat_features=cat_features)
    valid_pool = catboost.Pool(data=valid[feature_columns], label=valid['y'], group_id=valid['group'], cat_features=cat_features)

    params = {
        'loss_function': 'YetiRank',
        'use_best_model': True,
        'one_hot_max_size': 300,
        'metric_period': 100,
    }
    model = catboost.CatBoost(params)
    model.fit(train_pool, eval_set=valid_pool)

    valid_all['pred'] = model.predict(valid_all[feature_columns])
    pred = valid_all.groupby(['user', 'item'])['pred'].max().reset_index()

    pred = pred.sort_values(by=['user', 'pred'], ascending=False).reset_index(drop=True).groupby('user')['item'].apply(lambda x: list(x)[:12]).reset_index()
    pred

    gt = transactions.query("week == 1").groupby('user')['item'].apply(list).reset_index().rename(columns={'item': 'gt'})
    merged = gt.merge(pred, on='user', how='left')
    merged['item'] = merged['item'].fillna('').apply(list)
    merged

    return mapk(merged['gt'], merged['item'])


In [6]:
import optuna

study = optuna.create_study(
    direction='maximize',
    study_name='cat',
)
study.optimize(objective, timeout=3600*12)
study.trials_dataframe().sort_values(by='value',ascending=False).reset_index(drop=True).to_csv('cat.csv', index=False)

[I 2022-04-08 21:33:10,246] A new study created in memory with name: cat
Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	test: 0.3122110	best: 0.3122110 (0)	total: 655ms	remaining: 10m 54s
100:	test: 0.4568857	best: 0.4568857 (100)	total: 1m 4s	remaining: 9m 38s
200:	test: 0.4675848	best: 0.4675848 (200)	total: 2m 9s	remaining: 8m 34s
300:	test: 0.4730176	best: 0.4730176 (300)	total: 3m 13s	remaining: 7m 29s
400:	test: 0.4751279	best: 0.4751279 (400)	total: 4m 17s	remaining: 6m 24s
500:	test: 0.4779300	best: 0.4779300 (500)	total: 5m 21s	remaining: 5m 19s
600:	test: 0.4792671	best: 0.4792671 (600)	total: 6m 24s	remaining: 4m 15s
700:	test: 0.4802790	best: 0.4802790 (700)	total: 7m 28s	remaining: 3m 11s
800:	test: 0.4813834	best: 0.4813834 (800)	total: 8m 32s	remaining: 2m 7s
900:	test: 0.4818352	best: 0.4818352 (900)	total: 9m 35s	remaining: 1m 3s
999:	test: 0.4824847	best: 0.4824847 (999)	total: 10m 38s	remaining: 0us

bestTest = 0.4824847043
bestIteration = 999



[I 2022-04-08 21:47:06,025] Trial 0 finished with value: 0.02915937977555017 and parameters: {'num_repurchase_week': 15, 'repurchase_rank': 48, 'base_cands_cooc_week': 10, 'base_cands_cooc_rank': 354, 'base_cands_same_week': 8, 'base_cands_same_rank': 378}. Best is trial 0 with value: 0.02915937977555017.


0:	test: 0.2844800	best: 0.2844800 (0)	total: 670ms	remaining: 11m 9s
100:	test: 0.4387866	best: 0.4387866 (100)	total: 1m 6s	remaining: 9m 53s
200:	test: 0.4481047	best: 0.4481047 (200)	total: 2m 12s	remaining: 8m 47s
300:	test: 0.4535321	best: 0.4535321 (300)	total: 3m 17s	remaining: 7m 39s
400:	test: 0.4567864	best: 0.4567864 (400)	total: 4m 22s	remaining: 6m 31s
500:	test: 0.4580802	best: 0.4580802 (500)	total: 5m 26s	remaining: 5m 25s
600:	test: 0.4597455	best: 0.4597455 (600)	total: 6m 30s	remaining: 4m 19s
700:	test: 0.4611349	best: 0.4611349 (700)	total: 7m 34s	remaining: 3m 13s
800:	test: 0.4610720	best: 0.4611349 (700)	total: 8m 38s	remaining: 2m 8s
900:	test: 0.4618888	best: 0.4618888 (900)	total: 9m 42s	remaining: 1m 4s
999:	test: 0.4632193	best: 0.4632193 (999)	total: 10m 46s	remaining: 0us

bestTest = 0.4632193413
bestIteration = 999



[I 2022-04-08 22:01:26,277] Trial 1 finished with value: 0.02941018261892679 and parameters: {'num_repurchase_week': 14, 'repurchase_rank': 270, 'base_cands_cooc_week': 10, 'base_cands_cooc_rank': 316, 'base_cands_same_week': 15, 'base_cands_same_rank': 24}. Best is trial 1 with value: 0.02941018261892679.


...

[I 2022-04-09 09:09:28,359] Trial 52 finished with value: 0.02959593646845785 and parameters: {'num_repurchase_week': 16, 'repurchase_rank': 273, 'base_cands_cooc_week': 3, 'base_cands_cooc_rank': 50, 'base_cands_same_week': 14, 'base_cands_same_rank': 16}. Best is trial 20 with value: 0.029649876442112956.


0:	test: 0.2808043	best: 0.2808043 (0)	total: 602ms	remaining: 10m 1s
100:	test: 0.4425401	best: 0.4425401 (100)	total: 59.3s	remaining: 8m 47s
200:	test: 0.4545156	best: 0.4545156 (200)	total: 1m 58s	remaining: 7m 49s
300:	test: 0.4588175	best: 0.4588175 (300)	total: 2m 56s	remaining: 6m 49s
400:	test: 0.4620498	best: 0.4620498 (400)	total: 3m 54s	remaining: 5m 50s
500:	test: 0.4643860	best: 0.4643860 (500)	total: 4m 52s	remaining: 4m 51s
600:	test: 0.4656225	best: 0.4656225 (600)	total: 5m 50s	remaining: 3m 52s
700:	test: 0.4672388	best: 0.4672388 (700)	total: 6m 47s	remaining: 2m 53s
800:	test: 0.4681915	best: 0.4681915 (800)	total: 7m 45s	remaining: 1m 55s
900:	test: 0.4686218	best: 0.4686218 (900)	total: 8m 42s	remaining: 57.4s
999:	test: 0.4691494	best: 0.4691494 (999)	total: 9m 39s	remaining: 0us

bestTest = 0.4691494292
bestIteration = 999



[I 2022-04-09 09:22:35,180] Trial 53 finished with value: 0.029507901458924908 and parameters: {'num_repurchase_week': 15, 'repurchase_rank': 325, 'base_cands_cooc_week': 4, 'base_cands_cooc_rank': 7, 'base_cands_same_week': 13, 'base_cands_same_rank': 48}. Best is trial 20 with value: 0.029649876442112956.


0:	test: 0.2828157	best: 0.2828157 (0)	total: 828ms	remaining: 13m 47s
100:	test: 0.4354340	best: 0.4354340 (100)	total: 1m 20s	remaining: 11m 52s
200:	test: 0.4458625	best: 0.4458625 (200)	total: 2m 39s	remaining: 10m 33s
300:	test: 0.4508541	best: 0.4508541 (300)	total: 3m 57s	remaining: 9m 12s
400:	test: 0.4539983	best: 0.4539983 (400)	total: 5m 16s	remaining: 7m 53s
500:	test: 0.4560859	best: 0.4560859 (500)	total: 6m 34s	remaining: 6m 33s
600:	test: 0.4576422	best: 0.4576422 (600)	total: 7m 52s	remaining: 5m 13s
700:	test: 0.4585013	best: 0.4585013 (700)	total: 9m 10s	remaining: 3m 54s
800:	test: 0.4595073	best: 0.4595073 (800)	total: 10m 28s	remaining: 2m 36s
900:	test: 0.4603557	best: 0.4603557 (900)	total: 11m 45s	remaining: 1m 17s
999:	test: 0.4610104	best: 0.4610104 (999)	total: 13m 2s	remaining: 0us

bestTest = 0.4610104371
bestIteration = 999



[I 2022-04-09 09:39:14,924] Trial 54 finished with value: 0.029760615149452926 and parameters: {'num_repurchase_week': 16, 'repurchase_rank': 239, 'base_cands_cooc_week': 1, 'base_cands_cooc_rank': 38, 'base_cands_same_week': 16, 'base_cands_same_rank': 113}. Best is trial 54 with value: 0.029760615149452926.


In [7]:
import pickle
with open("study.pkl", 'wb') as f:
    pickle.dump(study, f)

In [8]:
optuna.visualization.plot_contour(study)

In [9]:
optuna.visualization.plot_param_importances(study)

In [10]:
optuna.visualization.plot_optimization_history(study)

In [11]:
df_params = study.trials_dataframe().sort_values(by='value',ascending=False).reset_index(drop=True)
df_params.head(10)

,number,value,datetime_start,datetime_complete,duration,params_base_cands_cooc_rank,params_base_cands_cooc_week,params_base_cands_same_rank,params_base_cands_same_week,params_num_repurchase_week,params_repurchase_rank,state
0,54,0.029761,2022-04-09 09:22:35.181683,2022-04-09 09:39:14.924512,0 days 00:16:39.742829,38,1,113,16,16,239,COMPLETE
1,20,0.029650,2022-04-09 01:46:02.974212,2022-04-09 01:59:40.506946,0 days 00:13:37.532734,152,1,55,14,16,317,COMPLETE
2,15,0.029633,2022-04-09 00:35:14.198013,2022-04-09 00:50:34.436641,0 days 00:15:20.238628,16,5,72,16,12,397,COMPLETE
3,32,0.029632,2022-04-09 04:22:21.044371,2022-04-09 04:36:35.773344,0 days 00:14:14.728973,67,5,38,15,15,292,COMPLETE
4,36,0.029630,2022-04-09 05:21:08.527101,2022-04-09 05:35:01.345800,0 days 00:13:52.818699,389,5,34,16,11,285,COMPLETE
5,31,0.029628,2022-04-09 04:09:01.583430,2022-04-09 04:22:21.043100,0 days 00:13:19.459670,36,5,26,16,15,291,COMPLETE
6,19,0.029598,2022-04-09 01:32:22.945884,2022-04-09 01:46:02.973001,0 days 00:13:40.027117,50,1,54,14,16,310,COMPLETE
7,52,0.029596,2022-04-09 08:58:27.988246,2022-04-09 09:09:28.359118,0 days 00:11:00.370872,50,3,16,14,16,273,COMPLETE
8,41,0.029563,2022-04-09 06:38:36.487205,2022-04-09 06:50:34.675355,0 days 00:11:58.188150,203,4,21,16,11,285,COMPLETE
9,48,0.029520,2022-04-09 08:01:54.987388,2022-04-09 08:14:17.705455,0 days 00:12:22.718067,102,8,18,14,15,206,COMPLETE
